# Tools and Agents using Ollama and LangChain

This notebook demonstrates building AI agents that can use external tools to answer questions:
- Creating custom tools (DuckDuckGo Search, Wikipedia)
- Building agents with tool-calling capabilities
- Using AgentExecutor for orchestration
- Real-world examples of information retrieval

## What are Agents?
Agents are LLM-powered systems that can decide which tools to use and when to use them based on user queries. They can chain multiple tool calls together to solve complex problems.

## Installation

In [ ]:
!pip3 install langchain
!pip3 install langchain-core
!pip3 install langchain-ollama
!pip3 install langchain_community

In [ ]:
!pip install -U ddgs
!pip install wikipedia

## Building Tools and Agents

### Import Required Components

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

### Create Agent Prompt Template

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Based on user query, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

### Initialize LLM

In [ ]:
llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0
)

## Example 1: Using DuckDuckGo and Wikipedia

### Create Tools

In [ ]:
search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

### Create Agent and Executor

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Example Query: How is Ollama used for running LLM locally?

In [ ]:
answer = agent_executor.invoke({"input": "How is Ollama used for running LLM locally"})
answer

## Example 2: Using Wikipedia Only

### Create Wikipedia-only Tool

In [ ]:
search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [wikipedia]

### Example Query: Who is Yann LeCun?

In [ ]:
answer = agent_executor.invoke({"input": "Who is Yann LeCun"})
answer

## How Agents Work

When you run an agent:

1. **Query Analysis**: The agent analyzes the user's question
2. **Tool Selection**: Decides which tool(s) to use (DuckDuckGo, Wikipedia, etc.)
3. **Tool Execution**: Calls the selected tool(s) with appropriate parameters
4. **Response Synthesis**: Combines tool outputs with its knowledge to generate a final answer
5. **Iteration**: Can make multiple tool calls if needed to answer complex questions

The `verbose=True` parameter shows the agent's reasoning process and tool calls in the output.

## Key Components

- **Tools**: External functions the agent can call (search engines, APIs, databases, etc.)
- **Agent**: The decision-making component powered by the LLM
- **AgentExecutor**: Orchestrates the agent's execution and tool calls
- **Prompt Template**: Defines the agent's behavior and instructions

## Benefits of Tool-Calling Agents

- Access to real-time information beyond training data
- Ability to perform actions (search, calculations, API calls)
- More accurate and up-to-date responses
- Flexible tool combinations for different use cases
- Transparent reasoning with verbose mode